In [1]:
contest_folder = 'practice'

In [2]:
from analysis import get_current_status, process_row

In [3]:
df_grouped, aggregated_df = get_current_status(contest_folder)

In [4]:
aggregated_list = aggregated_df.to_dict(orient='records')

In [5]:
aggregated_df

""


In [ ]:
import time
import threading

while True:
    _, aggregated_df = get_current_status(contest_folder)
    if len(aggregated_df) == 0:
        print("Not outputs found yet")
        time.sleep(30)
        continue
    
    aggregated_list = aggregated_df.to_dict(orient='records')
    print(f"Running analysis at {aggregated_df['timestring'][0]}")
    
    for row in aggregated_list:
        if row["status"] != "not_analyzed":
            continue
        analysis_thread = threading.Thread(target=process_row, args=(row,))
        analysis_thread.start()
        print(f"Running analysis at {row['timestring']} for {row['problem_name']}")

    time.sleep(30)

Not outputs found yet
Running analysis at 4505
Running analysis at 4505 for Fall in Line
Running analysis at 4505 for Line by Line
Running analysis at 4505 for Line of Delivery (Part 2)
Running analysis at 4535
Running analysis at 4535 for Fall in Line
Running analysis at 4535 for Line of Delivery (Part 2)
Running analysis at 4535 for Walk the Line
Running analysis at 4605
Running analysis at 4605 for Fall in Line
Running analysis at 4605 for Line of Delivery (Part 2)
Running analysis at 4635
Running analysis at 4635 for Line of Delivery (Part 1)
